In [ ]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

In [ ]:
# Load the trained model
model1 = load_model('../models/best_age_model.keras')
model2 = load_model('../models/best_emotion_model.keras')
model3 = load_model('../models/best_gender_model.keras')

In [ ]:
# Load a pre-trained face detector (e.g., Haar cascades)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
# Gender label dictionary
gender_dict = {0: 'Male', 1: 'Female'}

In [ ]:
# Constants
IDEAL_FACE_SIZE = 128  # Ideal size of the face in pixels
TOLERANCE = 30  # Tolerance for face size (can be adjusted)
STABLE_TIME_REQUIRED = 5  # Seconds required to be stable in the green square

In [ ]:
# Variables to track timing
start_time = None
predicted = False
final_pred_gender = None
final_pred_age = None

In [ ]:
def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        (x, y, w, h) = faces[0]  # Assuming the first detected face is the target
        face = gray[y:y+h, x:x+w]  # Crop the face
    else:
        face = gray  # Use the whole frame if no face is detected
    
    face = cv2.resize(face, (128, 128))  # Resize to model input size
    face = face / 255.0  # Normalize
    face = np.expand_dims(face, axis=-1)  # Add channel dimension
    face = np.expand_dims(face, axis=0)  # Add batch dimension
    
    return face, (x, y, w, h) if len(faces) > 0 else None

def predict_gender_age(img):
    processed_img, face_bbox = preprocess_image(img)
    pred = model.predict(processed_img)
    
    pred_gender_prob = pred[0][0].item()  # Extract scalar using item()
    pred_gender = gender_dict[round(pred_gender_prob)]  # Round to nearest gender
    
    pred_age_value = pred[1][0].item()  # Extract scalar using item()

    # Determine dynamic range for age prediction
    range_width = max(2, int(0.1 * pred_age_value))  # 10% of the predicted age, with a minimum range of 2
    pred_age_lower = max(0, round(pred_age_value - range_width))  # Lower bound
    pred_age_upper = round(pred_age_value + range_width)  # Upper bound
    
    return pred_gender, (pred_age_lower, pred_age_upper), face_bbox

In [ ]:
# Open a connection to the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    pred_gender, (pred_age_lower, pred_age_upper), face_bbox = predict_gender_age(frame)
    
    # Define the ideal square frame size and position
    frame_height, frame_width, _ = frame.shape
    square_size = 250  # Size of the square in pixels
    square_x = (frame_width - square_size) // 2
    square_y = (frame_height - square_size) // 2
    
    # Determine color of the square frame
    color = (0, 0, 255)  # Default to red (too far or too close)
    if face_bbox is not None:
        _, _, w, h = face_bbox
        face_size = max(w, h)
        
        if IDEAL_FACE_SIZE - TOLERANCE <= face_size <= IDEAL_FACE_SIZE + TOLERANCE:
            color = (0, 255, 0)  # Green (perfect distance)
            if start_time is None:
                start_time = time.time()  # Start timing when the face is first in the green square
            else:
                elapsed_time = time.time() - start_time
                if elapsed_time >= STABLE_TIME_REQUIRED:
                    predicted = True
                    final_pred_gender = pred_gender
                    final_pred_age = f'{pred_age_lower}-{pred_age_upper}'
        else:
            start_time = None  # Reset timing if the face leaves the green square
    else:
        start_time = None  # Reset timing if no face is detected
    
    # Draw the square frame
    cv2.rectangle(frame, (square_x, square_y), (square_x + square_size, square_y + square_size), color, 2)
    
    # Display real-time predictions or final predictions
    font_scale = 0.8  # Adjust font scale to make text smaller
    font_thickness = 2
    text_color = (255, 255, 255)  # White color for the text

    if predicted:
        cv2.putText(frame, f'Gender: {final_pred_gender}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)
        cv2.putText(frame, f'Age: {final_pred_age}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)
    else:
        cv2.putText(frame, f'Gender: {pred_gender}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)
        cv2.putText(frame, f'Age: {pred_age_lower}-{pred_age_upper}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)
    
    # Display the frame
    cv2.imshow('Gender and Age Prediction', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close windows
cap.release()
cv2.destroyAllWindows()